In [1]:
from pyiron_workflow import Workflow
import numpy as np
import matplotlib.pyplot as plt
import pyiron_workflow as pwf

In [2]:
from calphy_nodes import RunFreeEnergy, InputClass

In [3]:
@Workflow.wrap.as_function_node()
def increment_temperature(temperature,  
                          increment):
    new_temperature = temperature + increment
    return new_temperature

In [4]:
@Workflow.wrap.as_macro_node
def WhileRunFreeEnergy(wf, inp, 
                       species: str, 
                       potential: str,
                       temperature: float,
                       increment: float,
                       max_temperature: float):
    wf.free_energy = RunFreeEnergy(inp,
                                  species,
                                  potential,
                                  temperature)
    #wf.increment_temperature = increment_temperature(wf.free_energy.outputs.temperature,
    #                                        increment)
    wf.free_energy.inputs.temperature = wf.free_energy.outputs.incremented_temp
    wf.condition = pwf.standard_nodes.LessThan(wf.free_energy.outputs.incremented_temp, 
                                               max_temperature)
    
    wf.switch = pwf.standard_nodes.If()
    wf.switch.inputs.condition = wf.condition
    
    wf.starting_nodes = [wf.free_energy]
    wf.free_energy >> wf.condition >> wf.switch
    wf.switch.signals.output.true >> wf.free_energy
    return wf.free_energy

In [5]:
inp = InputClass(temperature=800,
                pressure=0,
                reference_phase = 'solid',
                mode = 'fe',
                equilibration_control = 'berendsen',
                )
inp.queue.cores = 4	

In [6]:
macro = WhileRunFreeEnergy(inp,
                          'Cu',
                          '2004--Zhou-X-W--Cu-Ag-Au--LAMMPS--ipr2',
                           800.,
                           100.,
                           1000.
                          )
#macro = RunFreeEnergy(inp,
#                          'Cu',
#                          '2004--Zhou-X-W--Cu-Ag-Au--LAMMPS--ipr2',
#                           800.,)

In [ ]:
macro.run()

/cmmc/ptmp/pyironhb/mambaforge/envs/pyiron_mpie_cmti_2024-09-09/lib/python3.11/site-packages/ase/io/lammpsdata.py:49: FutureWarning: "style" is deprecated; please use "atom_style".
  warnings.warn(
--------------------------------------------------------------------------

  Local host:   cmti001
  Local device: hfi1_0
--------------------------------------------------------------------------
[cmti001:25977] 3 more processes have sent help message help-mpi-btl-openib.txt / error in device init
[cmti001:25977] Set MCA parameter "orte_base_help_aggregate" to 0 to see all help / error messages
--------------------------------------------------------------------------

  Local host:   cmti001
  Local device: hfi1_0
--------------------------------------------------------------------------
[cmti001:26095] 3 more processes have sent help message help-mpi-btl-openib.txt / error in device init
[cmti001:26095] Set MCA parameter "orte_base_help_aggregate" to 0 to see all help / error messages


In [24]:
@Workflow.wrap.as_macro_node
def AddWhileLessThan(wf, a, b, cap):
    """
    Add :param:`b` to :param:`a` while the sum is less than or equal to :param:`cap`.

    A simple but complete demonstrator for how to construct cyclic flows, including
    logging key outputs during the loop.
    """
    # Bespoke logic
    wf.body = pwf.standard_nodes.Add(obj=a, other=b)
    wf.body.inputs.obj = wf.body.outputs.add  # Higher priority connection
    # The output is NOT_DATA on the first pass and `a` gets used, 
    # But after that the node will find and use its own output
    wf.condition = pwf.standard_nodes.LessThan(wf.body, cap)

    # Universal logic
    wf.switch = pwf.standard_nodes.If()
    wf.switch.inputs.condition = wf.condition

    wf.starting_nodes = [wf.body]
    wf.body >> wf.condition >> wf.switch
    wf.switch.signals.output.true >> wf.body
    return wf.body

In [25]:
macro = AddWhileLessThan(2,3,10)

In [26]:
macro.run()

{'body': 11}